In [1614]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [1615]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [1616]:
def put_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 19
        else:
            return 24
    else:
        return Age

In [1617]:
class1male = int(train_data[(train_data["Pclass"]==1) & (train_data["Sex"]=="male")]["Age"].mean())
class2male = int(train_data[(train_data["Pclass"]==2) & (train_data["Sex"]=="male")]["Age"].mean())
class3male = int(train_data[(train_data["Pclass"]==3) & (train_data["Sex"]=="male")]["Age"].mean())

class1female = int(train_data[(train_data["Pclass"]==1) & (train_data["Sex"]=="female")]["Age"].mean())
class2female = int(train_data[(train_data["Pclass"]==2) & (train_data["Sex"]=="female")]["Age"].mean())
class3female = int(train_data[(train_data["Pclass"]==3) & (train_data["Sex"]=="female")]["Age"].mean())
def set_age(cols):
    pclass = cols[0]
    sex = cols[1]
    age = cols[2]
    if pd.isnull(age):
        if pclass == 1:
            if sex == "male":
                return class1male
            else:
                return class1female
        elif pclass == 2:
            if sex == "male":
                return class2male
            else:
                return class2female
        else:
            if sex == "male":
                return class3male
            else:
                return class3female
    else:
        return age

In [1618]:
# train_data["Age"] = train_data[["Pclass", "Sex", "Age"]].apply(set_age, axis=1)
# test_data["Age"] = test_data[["Pclass", "Sex", "Age"]].apply(set_age, axis=1)

train_data["Age"] = train_data[["Age", "Pclass"]].apply(put_age, axis=1)
test_data["Age"] = test_data[["Age", "Pclass"]].apply(put_age, axis=1)

In [1619]:
train_data.drop("Cabin", axis=1, inplace=True)
test_data.drop("Cabin", axis=1, inplace=True)

In [1620]:
train_data["Embarked"] = train_data["Embarked"].apply(lambda x: "S" if pd.isnull(x) else x)

In [1621]:
fare = int(train_data[(train_data["Pclass"]==3) & (train_data["Sex"]=="male")]["Fare"].mean())
test_data["Fare"] = test_data["Fare"].apply(lambda x: fare if pd.isnull(x) else x)

In [1622]:
train_data['Title'] = train_data["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
train_data["Title"].value_counts()
test_data['Title'] = test_data["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
test_data["Title"].value_counts()

Mr        240
Miss       78
Mrs        72
Master     21
Col         2
Rev         2
Ms          1
Dona        1
Dr          1
Name: Title, dtype: int64

In [1623]:
train_data['Title'] = train_data['Title'].replace('Mlle', 'Miss')
train_data['Title'] = train_data['Title'].replace('Ms', 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')
train_data['Title'] = train_data['Title'].replace('Major', 'Military')
train_data['Title'] = train_data['Title'].replace('Col', 'Military')
train_data['Title'] = train_data['Title'].replace('Capt', 'Military')
train_data['Title'] = train_data['Title'].replace('Lady', 'Royal')
train_data['Title'] = train_data['Title'].replace('Don', 'Royal')
train_data['Title'] = train_data['Title'].replace('Countess', 'Royal')
train_data['Title'] = train_data['Title'].replace('Jonkheer', 'Royal')
train_data['Title'] = train_data['Title'].replace('Sir', 'Royal')
train_data['Title'] = train_data['Title'].replace('Dona', 'Royal')
train_data["Title"].value_counts()

test_data['Title'] = test_data['Title'].replace('Mlle', 'Miss')
test_data['Title'] = test_data['Title'].replace('Ms', 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')
test_data['Title'] = test_data['Title'].replace('Major', 'Military')
test_data['Title'] = test_data['Title'].replace('Col', 'Military')
test_data['Title'] = test_data['Title'].replace('Capt', 'Military')
test_data['Title'] = test_data['Title'].replace('Lady', 'Royal')
test_data['Title'] = test_data['Title'].replace('Don', 'Royal')
test_data['Title'] = test_data['Title'].replace('Countess', 'Royal')
test_data['Title'] = test_data['Title'].replace('Jonkheer', 'Royal')
test_data['Title'] = test_data['Title'].replace('Sir', 'Royal')
test_data['Title'] = test_data['Title'].replace('Dona', 'Royal')
test_data["Title"].value_counts()

Mr          240
Miss         79
Mrs          72
Master       21
Military      2
Rev           2
Dr            1
Royal         1
Name: Title, dtype: int64

In [1624]:
titanic=pd.concat([train_data, test_data], sort=False)
len_train=train_data.shape[0]
titanic['LastName'] = titanic["Name"].apply(lambda x: x.split(',')[0].strip())

In [1625]:
# sex = pd.get_dummies(train_data["Sex"], drop_first=True)
# embark = pd.get_dummies(train_data["Embarked"], drop_first=True)
# pclass = pd.get_dummies(train_data["Pclass"], drop_first=True)
# title = pd.get_dummies(train_data["Title"], drop_first=True)
# last = pd.get_dummies(train_data["LastName"], drop_first=True)
# train_data = pd.concat([train_data, sex, embark, pclass, title, last], axis=1)

# train_data.drop(["PassengerId", "Sex", "Embarked", "Name", "Ticket", "Pclass", "Title", "LastName"], axis=1, inplace=True)

# sex = pd.get_dummies(test_data["Sex"], drop_first=True)
# embark = pd.get_dummies(test_data["Embarked"], drop_first=True)
# pclass = pd.get_dummies(test_data["Pclass"], drop_first=True)
# title = pd.get_dummies(test_data["Title"], drop_first=True)
# last = pd.get_dummies(test_data["LastName"], drop_first=True)
# test_data = pd.concat([test_data, sex, embark, pclass, title, last], axis=1)

# test_data.drop(["PassengerId", "Sex", "Embarked", "Name", "Ticket", "Pclass", "Title", "LastName"], axis=1, inplace=True)

sex = pd.get_dummies(titanic["Sex"], drop_first=True)
embark = pd.get_dummies(titanic["Embarked"], drop_first=True)
pclass = pd.get_dummies(titanic["Pclass"], drop_first=True)
title = pd.get_dummies(titanic["Title"], drop_first=True)
last = pd.get_dummies(titanic["LastName"], drop_first=True)
titanic = pd.concat([titanic, sex, embark, pclass, title, last], axis=1)

titanic.drop(["PassengerId", "Sex", "Embarked", "Name", "Ticket", "Pclass", "Title", "LastName"], axis=1, inplace=True)

In [1626]:
train_data = titanic[:len_train]
test_data = titanic[len_train:]
train_data["Survived"] = train_data["Survived"].astype("int")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1627]:
# def has_family(data):
#     if data[0] > 0 or data[1] > 0:
#         return 1
#     else:
#         return 0

# train_data["family"] = train_data[["SibSp", "Parch"]].apply(has_family, axis=1)
# train_data["family"] = train_data["SibSp"] + train_data["Parch"]
# test_data.drop(["SibSp", "Parch"], axis=1, inplace=True)

In [1628]:
train_data.isnull().sum()[train_data.isnull().sum()>0]
test_data.isnull().sum()[test_data.isnull().sum()>0]
test_data

,Survived,Age,SibSp,Parch,Fare,male,Q,S,2,3,...,Zabour,Zakarian,Zimmerman,de Brito,de Messemaeker,de Mulder,de Pelsmaeker,del Carlo,van Billiard,van Melkebeke
0,NaN,34.5,0,0,7.8292,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,NaN,47.0,1,0,7.0000,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,NaN,62.0,0,0,9.6875,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,27.0,0,0,8.6625,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,NaN,22.0,1,1,12.2875,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,24.0,0,0,8.0500,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
414,NaN,39.0,0,0,108.9000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,NaN,38.5,0,0,7.2500,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
416,NaN,24.0,0,0,8.0500,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [1629]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# X = train_data.drop("Survived", axis=1)
# y = train_data["Survived"]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

# X_train = X
# y_train = y

# X_test = X
# y_test = y

# model = RandomForestClassifier(random_state = 1)
# model.fit(X_train, y_train)
# predictions = model.predict(X_test)

# acc_random_forest = round(model.score(X_train, y_train) * 100, 2)
# print("train", acc_random_forest)
# acc_random_forest = round(model.score(X_test, y_test) * 100, 2)
# print("test ", acc_random_forest)

# print(classification_report(y_test, predictions))
# print(confusion_matrix(y_test, predictions))

train 99.89
test  99.89
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       549
           1       1.00      1.00      1.00       342

    accuracy                           1.00       891
   macro avg       1.00      1.00      1.00       891
weighted avg       1.00      1.00      1.00       891

[[548   1]
 [  0 342]]


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

xtrain = train_data.drop("Survived",axis=1)
ytrain = train_data['Survived']
xtest = test_data.drop("Survived", axis=1)

svc = make_pipeline(StandardScaler(),SVC(random_state=1))
r = [0.0001,0.001,0.1,1,10,50,100]
PSVM = [{'svc__C':r, 'svc__kernel':['linear']},
      {'svc__C':r, 'svc__gamma':r, 'svc__kernel':['rbf']}]
GSSVM = GridSearchCV(estimator=svc, param_grid=PSVM, scoring='accuracy', cv=2)
scores_svm = cross_val_score(GSSVM, xtrain.astype(float), ytrain,scoring='accuracy', cv=5)
model = GSSVM.fit(xtrain, ytrain)
np.mean(scores_svm)

In [1630]:
predictions = model.predict(test_data.drop("Survived", axis=1))

test_dataID = pd.read_csv("/kaggle/input/titanic/test.csv")
output = pd.DataFrame({'PassengerId': test_dataID.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
